<a href="https://colab.research.google.com/github/librairy/bio-nlp/blob/master/notebooks/diseases/mesh_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 

data = pd.read_csv(
    "data/diseases.csv",      
    sep=',',
    header=28, 
    skipinitialspace=True,
    quotechar="\""
)

# Preview the first 5 lines of the loaded data 
data.head()


,DiseaseName,DiseaseID,AltDiseaseIDs,Definition,ParentIDs,TreeNumbers,ParentTreeNumbers,Synonyms,SlimMappings
0,10p Deletion Syndrome (Partial),MESH:C538288,NaN,NaN,MESH:D002872|MESH:D025063,C16.131.260/C538288|C16.320.180/C538288|C23.55...,C16.131.260|C16.320.180|C23.550.210.050.500.500,"Chromosome 10, 10p- Partial|Chromosome 10, mon...",Congenital abnormality|Genetic disease (inborn...
1,13q deletion syndrome,MESH:C535484,NaN,NaN,MESH:D002872|MESH:D025063,C16.131.260/C535484|C16.320.180/C535484|C23.55...,C16.131.260|C16.320.180|C23.550.210.050.500.500,Chromosome 13q deletion|Chromosome 13q deletio...,Congenital abnormality|Genetic disease (inborn...
2,15q24 Microdeletion,MESH:C579849,DO:DOID:0060395,NaN,MESH:D002872|MESH:D008607|MESH:D025063,C10.597.606.360/C579849|C16.131.260/C579849|C1...,C10.597.606.360|C16.131.260|C16.320.180|C23.55...,15q24 Deletion|15q24 Microdeletion Syndrome|In...,Congenital abnormality|Genetic disease (inborn...
3,16p11.2 Deletion Syndrome,MESH:C579850,NaN,NaN,MESH:D001321|MESH:D002872|MESH:D008607|MESH:D0...,C10.597.606.360/C579850|C16.131.260/C579850|C1...,C10.597.606.360|C16.131.260|C16.320.180|C23.55...,NaN,Congenital abnormality|Genetic disease (inborn...
4,"17,20-Lyase Deficiency, Isolated",MESH:C567076,NaN,NaN,MESH:D000312,C12.706.316.090.500/C567076|C13.351.875.253.09...,C12.706.316.090.500|C13.351.875.253.090.500|C1...,"17-Alpha-Hydroxylase-17,20-Lyase Deficiency, C...",Congenital abnormality|Endocrine system diseas...


In [ ]:
import pysolr
import math

solr = pysolr.Solr('http://librairy.linkeddata.es/data/diseases', timeout=2)

updated_drugs = []
num = 0;
diseases = []
for row in data.itertuples(index=False):
    if (num % 500 == 0 and num > 0):
        solr.add(diseases)
        solr.commit()
        diseases = []
        print(num, "diseases procesed!")
    document = {}
    document['name_t']=row[0]
    id_value = row[1].split(':')[1]
    if (id_value.isdecimal()):
        id_value = "X"+str(row[1].split(':')[1])
    document['id']=id_value
    if (isinstance(row[4], str)):
        document['parent']=[d.split(':')[1] for d in row[4].split('|')]
    if (isinstance(row[6], str)):
        document['level_i']=len(row[6].split('|'))
    if (isinstance(row[7], str)):
        official_synonyms = [d for d in row[7].split('|')]
        for synonym in official_synonyms:
            if ("," in synonym):
                syn_tokens =[s.strip() for s in synonym.split(",")]
                syn_tokens.reverse()
                critical_synonym = " ".join(syn_tokens)
                official_synonyms.append(critical_synonym)
        document['synonyms']= official_synonyms
    if (isinstance(row[8], str)):
        document['mappings']=[d for d in row[8].split('|')]
    #print(document)
    diseases.append(document)
    num+=1
print(num,"diseases indexed")    

500 diseases procesed!
1000 diseases procesed!
1500 diseases procesed!
2000 diseases procesed!
2500 diseases procesed!
3000 diseases procesed!
3500 diseases procesed!
4000 diseases procesed!
4500 diseases procesed!
5000 diseases procesed!
5500 diseases procesed!
6000 diseases procesed!
6500 diseases procesed!
7000 diseases procesed!
7500 diseases procesed!
8000 diseases procesed!
8500 diseases procesed!
9000 diseases procesed!
9500 diseases procesed!
10000 diseases procesed!
10500 diseases procesed!
11000 diseases procesed!
11500 diseases procesed!
12000 diseases procesed!
12500 diseases procesed!
13000 diseases procesed!
13059 diseases indexed
